# Match Trajectories using FMM
#### Jalal Khalil (jalalk@uab.edu)

**Input**: raw trajectories around Birmingham, AL, USA\
**Output**: matched trajectories

**Steps**:
- Get the raw trajectories
- Install FMM
- Prepare the network
- Use FMM for matching
- Plot the results

### Get the raw data

In [1]:
# download file from GitHub
! wget https://raw.githubusercontent.com/jalal1/fmm_jupyter/master/data/100_trajectories.txt

--2022-08-17 23:06:28--  https://raw.githubusercontent.com/jalal1/fmm_jupyter/master/data/100_trajectories.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 25479 (25K) [text/plain]
Saving to: ‘100_trajectories.txt.1’

100_trajectories.tx 100%[===================>]  24.88K  --.-KB/s    in 0s      

2022-08-17 23:06:28 (58.1 MB/s) - ‘100_trajectories.txt.1’ saved [25479/25479]



In [2]:
!head /content/100_trajectories.txt

Participant2_2019_March2019_Shot0140.jpg:33.509364058739486 -86.78954869584962,33.50978487475548 -86.78860322799623,33.51365016246786 -86.792410503766
Participant2_2019_May2019_Shot0142.jpg:33.51094614347225 -86.78923299192616,33.509571534630986 -86.78998517826301,33.51938097370663 -86.79837386198179,33.50822037079172 -86.82131054613122,33.51100887200205 -86.82332334728943,33.51131870158742 -86.83317062553503
Participant2_2019_November2019_Shot0127.jpg:33.50679887410579 -86.80424681374738,33.51400856087657 -86.80915731051758,33.51335332100002 -86.81056606651244,33.51288259528744 -86.81025033563954
Participant2_2019_November2019_Shot0198.jpg:33.51188921624417 -86.78723301871021,33.509581833511554 -86.79156995015465,33.48366945634628 -86.78709029437786,33.4661537869968 -86.76220036737358
Participant2_2019_November2019_Shot0267.jpg:33.49995290121026 -86.79772386799166,33.50518863176492 -86.80128439451919,33.51110099859621 -86.78887415090277,33.51283854022087 -86.79006323287805
Participant

In [3]:
import os
raw_trajectories = []

with open('/content/100_trajectories.txt', 'r') as f:
    for line in f:
      line_values = line.split(":")
      traj = line_values[1]
      raw_trajectory = []
      for x in traj.split(','):
        lat, lon = x.split(" ")
        raw_trajectory.append((float(lat), float(lon)))
      raw_trajectories.append(raw_trajectory)

print("Number of trajectories: ", len(raw_trajectories))

Number of trajectories:  100


### Install Fast map matching (FMM)*

Reference: Can Yang & Gyozo Gidofalvi (2018) Fast map matching, an algorithm
integrating hidden Markov model with precomputation, International Journal of Geographical Information Science, 32:3, 547-570, DOI: 10.1080/13658816.2017.1400548

https://fmm-wiki.github.io/

As you may have noticed, the trajectories are not aligned with the underlying road network. i.e., a line between two consecutive points is a straight line and does not follow the actual roads.

To do match those points to the road network, and add back intermediate points, we use FMM. First, we need to install it.

**Install Prerequisites**

In [4]:
### TODO ###

# install FMM "requirements" following https://fmm-wiki.github.io/docs/installation
# You need to choose the right OS on that page
# For example, on Ubuntu you can run
# !sudo apt-get install libboost-dev libboost-serialization-dev gdal-bin libgdal-dev make cmake libbz2-dev libexpat1-dev swig python-dev

In [5]:
# Install all the requirements with:
! sudo apt-get install libboost-dev libboost-serialization-dev \
gdal-bin libgdal-dev make cmake libbz2-dev libexpat1-dev swig python-dev

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libboost-dev is already the newest version (1.65.1.0ubuntu1).
make is already the newest version (4.1-9.1ubuntu1).
python-dev is already the newest version (2.7.15~rc1-1).
gdal-bin is already the newest version (2.2.3+dfsg-2).
libboost-serialization-dev is already the newest version (1.65.1.0ubuntu1).
libgdal-dev is already the newest version (2.2.3+dfsg-2).
swig is already the newest version (3.0.12-1).
cmake is already the newest version (3.10.2-1ubuntu2.18.04.2).
libbz2-dev is already the newest version (1.0.6-8.1ubuntu0.2).
libexpat1-dev is already the newest version (2.2.5-3ubuntu0.7).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 20 not upgraded.


**Install FMM**

In [6]:
!git clone https://github.com/cyang-kth/fmm.git

fatal: destination path 'fmm' already exists and is not an empty directory.


In [7]:
# !!!! This could take ~ 6 mintues !!!!
# it may not work and requires for password, etc.
# in this case, go to your folder using console to compile and install FMM

import os
# change working directory
os.chdir("fmm")

if not os.path.exists('build'):
  os.mkdir('build')
# ! mkdir build
os.chdir("build")
# ! cd build
! cmake ..
! make -j4
! sudo make install

-- CMAKE version 3.22.6
-- Set CMP0074 state to NEW
-- Set CMP0086 state to NEW
-- Set CMP0078 state to NEW
-- 
No conda environment found in PATH!
PATH=/opt/bin:/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin

-- Could NOT find Conda (missing: CONDA_PREFIX) 
-- Non conda exist, search library in default path
-- GDAL headers found at /usr/include/gdal
-- GDAL library found at /usr/lib/libgdal.so
-- Boost headers found at /usr/include
-- Boost library found at /usr/lib/x86_64-linux-gnu/libboost_serialization.so
-- Boost library version 1_65_1
-- OpenMP_HEADERS found at 
-- OpenMP_CXX_LIBRARIES found at /usr/lib/gcc/x86_64-linux-gnu/7/libgomp.so;/usr/lib/x86_64-linux-gnu/libpthread.so
-- Installation folder /usr/local
-- Not install fmm headers
-- Add python cmake information
-- Swig version is 3.0.12
-- Python header found at /usr/include/python3.7m
-- Python library found at /usr/lib/x86_

In [8]:
# Verfication of installation
!fmm

[info][fmm_app_config.cpp:49 ] Start reading FMM configuration from arguments
fmm argument lists:
--ubodt (required) <string>: Ubodt file name
--network (required) <string>: Network file name
--network_id (optional) <string>: Network id name (id)
--source (optional) <string>: Network source name (source)
--target (optional) <string>: Network target name (target)
--gps (required) <string>: GPS file name
--gps_id (optional) <string>: GPS id name (id)
--gps_x (optional) <string>: GPS x name (x)
--gps_y (optional) <string>: GPS y name (y)
--gps_timestamp (optional) <string>: GPS timestamp name (timestamp)
--gps_geom (optional) <string>: GPS geometry name (geom)
--gps_point (optional): if specified read input data as gps point, otherwise (default) read input data as trajectory
--output (required) <string>: Output file name
--output_fields (optional) <string>: Output fields
  opath,cpath,tpath,mgeom,pgeom,
  offset,error,spdist,tp,ep,length,duration,speed,all
-k/--candidates (optional) <int>

In [9]:
# Change to the parent folder which contains fmm_test.py
if os.getcwd() != "/content/fmm/example/python":
  os.chdir("/content/fmm/example/python")
os.system('python fmm_test.py')

256

In [10]:
# Install osmnx package to prepare the network, that will be used as input to FMM
!pip install osmnx

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### Steps to run FMM after installation: 
1- Prepare the network using osmnx \
2- Run the match(...) function

In [11]:
# !!!! This could take ~ 2 mintues !!!!

import osmnx as ox
import time
from shapely.geometry import Polygon
import os
import numpy as np
from fmm import Network,NetworkGraph,STMATCH,STMATCHConfig

def save_graph_shapefile_directional(G, filepath=None, encoding="utf-8"):
    # default filepath if none was provided
    if filepath is None:
        filepath = os.path.join(ox.settings.data_folder, "graph_shapefile")

    # if save folder does not already exist, create it (shapefiles
    # get saved as set of files)
    if not filepath == "" and not os.path.exists(filepath):
        os.makedirs(filepath)
    filepath_nodes = os.path.join(filepath, "nodes.shp")
    filepath_edges = os.path.join(filepath, "edges.shp")

    # convert undirected graph to gdfs and stringify non-numeric columns
    gdf_nodes, gdf_edges = ox.utils_graph.graph_to_gdfs(G)
    gdf_nodes = ox.io._stringify_nonnumeric_cols(gdf_nodes)
    gdf_edges = ox.io._stringify_nonnumeric_cols(gdf_edges)
    # We need an unique ID for each edge
    gdf_edges["fid"] = np.arange(0, gdf_edges.shape[0], dtype='int')
    # save the nodes and edges as separate ESRI shapefiles
    gdf_nodes.to_file(filepath_nodes, encoding=encoding)
    gdf_edges.to_file(filepath_edges, encoding=encoding)

print("osmnx version",ox.__version__)

# !!! Download by a bounding box # !!!
# -------------------------------------------------
bounds = (-86.9671,-86.5901,33.3472,33.6598)
# -------------------------------------------------
x1,x2,y1,y2 = bounds
boundary_polygon = Polygon([(x1,y1),(x2,y1),(x2,y2),(x1,y2)])
G = ox.graph_from_polygon(boundary_polygon, network_type='drive')
start_time = time.time()
save_graph_shapefile_directional(G, filepath='./network-new')
print("--- %s seconds ---" % (time.time() - start_time))

osmnx version 1.1.2


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.


--- 28.58013939857483 seconds ---


In [12]:
# copy network folder to /content/fmm/example/python/network-new
!mv '/content/network-new' '/content/fmm/example/python/network-new'

mv: cannot stat '/content/network-new': No such file or directory


In [13]:
# match(...) function

def match(path,points):
  network = Network(path,"fid","u","v")
  graph = NetworkGraph(network)
  print (graph.get_num_vertices())
  model = STMATCH(network,graph)
  wkt  = str(points)
  config = STMATCHConfig()
  config.k = 4
  config.gps_error = 0.5
  config.radius = 0.4
  config.vmax = 30;
  config.factor =1.5
  result = model.match_wkt(wkt,config)
  print (type(result))
  # print ("Opath ",list(result.opath))
  # print ("Cpath ",list(result.cpath))
  # print ("WKT ",result.mgeom.export_wkt())
  return result.mgeom.export_wkt()

In [14]:
def match_trajs(network_path, traj):
  """
  match [traj] on [network_path]. [traj] should be [(lon1, lat1), (lon2, lat2), ...]
  Output: matched traj as "LINESTRING(-86.797211 33.499194, -86.7973..."
  """
  from shapely.geometry import LineString
  # import geopandas !!

  traj_ = LineString(traj)
  # print(traj_)
  return match(network_path, traj_)

In [15]:
import csv
from shapely.geometry import Point
from collections import defaultdict

edges_path = '/content/fmm/example/python/network-new/edges.shp'

matched_trajectories = []

# !! FMM the first x values !! 
x = 10

for traj in raw_trajectories[:x]:
  # print(img_id, traj)
  # swap (lat, lon) --> (lon, lat) as FMM requires lon before lat
  traj_m = [(sub[1], sub[0]) for sub in traj]
  fmm_traj = match_trajs('/content/fmm/example/python/network-new/edges.shp', traj_m)
  # remove LINESTRING and extra parentheses
  fmm_traj = fmm_traj.split("LINESTRING(")[1]
  fmm_traj = fmm_traj.split(")")[0]
  traj_list = []
  for lon_lat in fmm_traj.split(','):
    lon__lat = lon_lat.split(" ")
    # print(lon__lat)
    if len(lon__lat) == 2:
      traj_list.append((float(lon__lat[1]), float(lon__lat[0])))
    else:
      print("!! lon and lat should be 2 !!", lon__lat)
  matched_trajectories.append(traj_list)

32332
<class 'fmm.PyMatchResult'>
32332
<class 'fmm.PyMatchResult'>
32332
<class 'fmm.PyMatchResult'>
32332
<class 'fmm.PyMatchResult'>
32332
<class 'fmm.PyMatchResult'>
32332
<class 'fmm.PyMatchResult'>
32332
<class 'fmm.PyMatchResult'>
32332
<class 'fmm.PyMatchResult'>
32332
<class 'fmm.PyMatchResult'>
32332
<class 'fmm.PyMatchResult'>


### Plot using Folium

In [18]:
! pip install folium

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [20]:
import random
import folium

map = folium.Map(zoom_start=13, height=700, location=[33.5186, -86.8104])

# choose random trajectory to plot
x = random.randrange(len(matched_trajectories))

# original trajectory
folium.PolyLine(raw_trajectories[x], color='red', radiuses=2, connect=True).add_to(map)
# matched trajectory
folium.PolyLine(matched_trajectories[x], color='green', radiuses=2, connect=True).add_to(map)

map

In [21]:
# plot trajectories
from folium.plugins import HeatMap

map = folium.Map(zoom_start=10, height=700, location=[33.5186, -86.8104])
for d in matched_trajectories:
  HeatMap(d, blur=5, radius=5).add_to(map)

map

In [22]:
# decompose trajectories into points and plot
trajs_as_points = []
for r in matched_trajectories:
  for rr in r:
    trajs_as_points.append(rr)

map = folium.Map(zoom_start=10, height=700, location=[33.5186, -86.8104])
HeatMap(trajs_as_points, blur=5, radius=5).add_to(map)
map

In [23]:
# The End